In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [3]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [4]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [5]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

C:\Users\elaim\miniconda3\envs\d2l\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(7.4506e-09, grad_fn=<MeanBackward0>)

In [7]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [8]:
linear = MyLinear(5, 3) # 给权重和偏置赋值
linear.weight

Parameter containing:
tensor([[-0.5330, -1.6268,  1.4759],
        [-0.2228, -0.3814, -0.5851],
        [-0.7432,  1.4242, -0.1002],
        [-0.4747, -0.4207,  0.1872],
        [-0.9578, -0.4291,  0.2885]], requires_grad=True)

In [9]:
linear(torch.rand(2, 5)) # 输入参数前向传播得到结果

tensor([[0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.7772]])

In [10]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.6436],
        [0.0000]])

In [ ]:
""" Exercise """

In [17]:
# 1.
import torch
import torch.nn as nn

class ReductionLayer(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ReductionLayer, self).__init__()
        # 第一个参数：当前类名，它告诉super从哪个类的角度开始查找父类
        # 第二个参数：当前类的实例。它告诉super这个方法或属性的调用是针对哪个具体实例的。
        # 在这里两个参数的作用略有重合，传两个参数的主要场景还是在多重继承里，这里只有单一继承传一个就够了。
        self.W = nn.Parameter(torch.randn(output_dim, input_dim, input_dim))
        # 实际上output_dim, input_dim, input_dim对应于k, i, j的维度
    
    def forward(self, x):
        # print(x)
        x_expanded = x.unsqueeze(1).expand(-1, self.W.shape[1], -1)
        # x.unsqueeze(1) 的作用是在 x 的第 1 维度上添加一个新维度。**添加新维度 1 **
        # expand 用于将张量在某些维度上进行扩展而不复制数据。 
        # 将 x 的第 1 维扩展到与 self.W.shape[1] 相同的大小，同时保持其他维度不变（-1 表示该维度不变）。
        # print(x_expanded)
        reduction_result = (self.W * x_expanded).sum(dim=(1, 2))
        # **逐元素相乘**，然后在 1 和 2 两个维度上求和，这里不是矩阵乘法
        return reduction_result
        

reduction_layer = ReductionLayer(input_dim=3, output_dim=2)
x = torch.tensor([[1.0, 2.0, 3.0],
                  [4.0, 5.0, 6.0]])
reduction_output = reduction_layer(x)
reduction_output


tensor([-8.6209, -6.2672], grad_fn=<SumBackward1>)

In [4]:
# 2.
import torch.fft

# Exercise 2: Define a layer that returns the leading half of Fourier coefficients
class FourierHalfLayer(nn.Module):
    def __init__(self):
        super(FourierHalfLayer, self).__init__()

    def forward(self, x):
        # Compute the Fourier transform
        fft_output = torch.fft.fft(x, dim=-1)
        # Return the leading half of the coefficients
        half_size = fft_output.shape[-1] // 2
        return fft_output[..., :half_size]

# Testing the FourierHalfLayer
fourier_layer = FourierHalfLayer()
x = torch.tensor([[1.0, 2.0, 3.0, 4.0], [4.0, 5.0, 6.0, 7.0]])  # Example input
fourier_output = fourier_layer(x)
fourier_output


tensor([[10.+0.j, -2.+2.j],
        [22.+0.j, -2.+2.j]])